In [3]:
%pip install -Uq transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2")

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [8]:
training_data = [
    {"question": "Привет, как тебя зовут?",
     "answer": "Привет, я чат-бот, разработанный для ответов на вопросы."},
    {"question": "Как дела?",
     "answer": "Всё хорошо, спасибо!"},
    {"question": "О чём ты любишь говорить?",
     "answer":"Обо всём!"},
    {"question": "Скажи, что такое искусственный интеллект?",
     "answer": "Искусственный интеллект (ИИ) — технология для создания интеллектуальных программ, которая имитирует человеческое поведение."},
    {"question": "Какие языки программирования поддерживаются чат-ботом?",
     "answer": "Чат-бот создан на Python и может взаимодействовать с пользователем на различных языках."},  
    {"question": "Что нового в мире технологий?",
     "answer": "Последние новости в мире технологий включают разработку искусственного интеллекта и электромобилей."},
    {"question": "Спасибо, чат-бот",
     "answer": "Чем я могу еще помочь?"},
    {"question": "Сколько месяцев в году?",
     "answer": " В году двенадцать месяцев"},
    {"question": "Как закрыть сессию?",
     "answer": "Чтобы закрыть сессию нужно ее закрыть"},

]

In [9]:
transformer_training_data = [
    {"question": "Привет, как тебя зовут?",
     "answer": "Привет, я чат-бот, разработанный для ответов на вопросы."},
    {"question": "Как дела?",
     "answer": "Всё хорошо, спасибо!"},
    {"question": "О чём ты любишь говорить?",
     "answer":"Обо всём!"},
    {"question": "Скажи, что такое искусственный интеллект?",
     "answer": "Искусственный интеллект (ИИ) — технология для создания интеллектуальных программ, которая имитирует человеческое поведение."},
    {"question": "Какие языки программирования поддерживаются чат-ботом?",
     "answer": "Чат-бот создан на Python и может взаимодействовать с пользователем на различных языках."},  
    {"question": "Что нового в мире технологий?",
     "answer": "Последние новости в мире технологий включают разработку искусственного интеллекта и электромобилей."},
    {"question": "Спасибо, чат-бот",
     "answer": "Чем я могу еще помочь?"},
    {"question": "Сколько месяцев в году?",
     "answer": " В году двенадцать месяцев"},
    {"question": "Как закрыть сессию?",
     "answer": "Чтобы закрыть сессию нужно ее закрыть"},

]

In [10]:
vectorizer = TfidfVectorizer()
corpus = [item['question'] for item in training_data]
tfidf_matrix = vectorizer.fit_transform(corpus)
print(tfidf_matrix.toarray())

[[0.         0.         0.         0.         0.         0.
  0.53154671 0.         0.         0.39035151 0.         0.
  0.         0.         0.         0.         0.53154671 0.
  0.         0.         0.         0.         0.         0.53154671
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.80600667 0.
  0.         0.         0.         0.59190645 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.5        0.         0.         0.
  0.         0.         0.         0.         0.         0.5
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.5        0.         0.         0.5        0.        ]
 [0.         0.         0.         0.         0.         0.
  0.

In [11]:

corpus = [item['question'] for item in transformer_training_data ]


In [26]:
def tfidf_get_response(user_question):
    user_tfidf = vectorizer.transform([user_question])
    similarity_scores = cosine_similarity(user_tfidf, tfidf_matrix) 
    max_similarity_index = np.argmax(similarity_scores)
    return training_data[max_similarity_index + 1]

def transform_generate_response(user_question):
    inputs = tokenizer.encode(user_question, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def hybrid_get_response(user_question):

    tfidf_response = tfidf_get_response(user_question)
    transform_response = transform_generate_response(user_question)

    if len(tfidf_response) > len(transform_response):
        return tfidf_response
    else:
        return transform_response

while True:
    user_question = input("Пользователь: ")
    if user_question.lower() in ['exit']:
        print("Чат-бот: До свидания!")
        break
    response = hybrid_get_response(user_question)
    print("Чат-бот:", response)

Чат-бот: До свидания!
